<h3 align="center" style="font-size: 35px; color: #800080; font-family: Georgia;">
    <span style="color: #008080;"> Author:</span> 
    <span style="color: black;">Ali Can PAYASLI</span>
</h3>

<div style="border-radius:10px;border:black solid;padding: 15px;background-color:white;font-size:110%;text-align:left">
<div style="font-family:Georgia;background-color:'#DEB887'; padding:30px; font-size:17px">

   
CONTENT:<br><br>
1. [Libraries](#1)<br>
2. [Data Load and General Overview](#2)<br>
3. [Visualize and GroupBy](#3)<br>
4. [Missing Values and Feature Engineering](#4)<br>
    * [Name and Cabin](#5)
    * [PassengerId](#6)
    * [Luxury](#7)
    * [HomePlanet](#8)
    * [Destination](#9)
    * [VIP](#10)
    * [Cryosleep](#11)
    * [Age](#12)
    * [C1-C2](#13)
    * [Surname and ID Drop](#14)
    * [Feature Converting](#15)
    * [Outlier Samples](#16)
    * [Normalize](#17)
    * [Train-Test Split for Return to original Dataset](#18)
5. [Model](#19)
    * [Dependent and Undependent Variable](#20)
    * [Train-Test Split For Model](#21)
    * [LGB](#22)

</div>
</div>

<a id = "1"></a>
# Libraries

In [ ]:
import numpy as np 
import pandas as pd 
import seaborn as sns
import missingno as mss
import matplotlib.pyplot as plt
import os
import warnings
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split,cross_val_score
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, ConfusionMatrixDisplay, confusion_matrix
import matplotlib.pyplot as plt
from sklearn.neural_network import MLPClassifier
from sklearn.linear_model import LogisticRegression
from lightgbm import LGBMClassifier
import lightgbm as lgb
warnings.filterwarnings("ignore")
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

In [ ]:

train_data = pd.read_csv("/kaggle/input/spaceship-titanic/train.csv")
test_data = pd.read_csv("/kaggle/input/spaceship-titanic/test.csv")

In [ ]:
train_data[:20]

In [ ]:
test_data

In [ ]:
train_data.info()

In [ ]:
train_data.describe().T

In [ ]:
train_data.describe(include = "all").T

In [ ]:
train_data["Transported"].value_counts()

In [ ]:
#True convert 1 false 0 for bool feature
train_data["Transported"] = [1 if i == True else 0 for i in train_data["Transported"]]

In [ ]:
train_data["Transported"].value_counts()

In [ ]:
train_data["PassengerId"].value_counts()

In [ ]:
train_data["HomePlanet"].value_counts()

In [ ]:
train_data["CryoSleep"].value_counts()

In [ ]:
len(train_data["Cabin"].unique())

In [ ]:
train_data["Destination"].value_counts()

In [ ]:
train_data["VIP"].value_counts()

In [ ]:
len(train_data["Name"].unique())

In [ ]:
train_data["Age"].describe()

In [ ]:
train_data["RoomService"].describe()

In [ ]:
train_data["FoodCourt"].describe()

In [ ]:
train_data["ShoppingMall"].describe()

In [ ]:
train_data["Spa"].describe()

In [ ]:
train_data["VRDeck"].describe()

<div style="border-radius:10px; border:#808080 solid; padding: 15px; background-color: ##F0E68C ; font-size:100%; text-align:left">

<h3 align="left"><font color=blue>Observe</font></h3>

* **Train dataset has 8693 samples and 14 features**
* **Test dataset has 4277 samples and 13 features**
* **Target feauture name Transported (True or False)**
* **Dataset has 13 Undependent features. These features 7 object and 6 numeric**
* **PassengerId has unique values**
* **HomePlanet has 3 different values and its name Earth, Europa, Mars**
* **CryoSleep has boolean values True or False**
* **Cabin features has too much different values**
* **Destination has 3 different values and its name TRAPPIST-1e, 55 Cancri e and PSO J318.5-22**
* **VIP has boolean values True or False**
* **Name feature has too much different values.**
* **Age feature min value 0 and max value 79, median 27 and mean 28.82**
* **RoomService, FoodCourt, ShoppingMall, Spa, VRDeck luxury amenitiens and therefore relationship VIP**
    
    


<a id = "3"></a>
# Visualize

In [ ]:
sns.barplot(x = "VIP", y = "Transported", data = train_data)

In [ ]:
train_data.groupby("Transported")["VIP"].value_counts()

In [ ]:
sns.barplot(x = "HomePlanet", y = "Transported", data = train_data)

In [ ]:
train_data.groupby("Transported")["HomePlanet"].value_counts()

In [ ]:
sns.barplot(x = "CryoSleep", y = "Transported", data = train_data)

In [ ]:
train_data.groupby("Transported")["CryoSleep"].value_counts()

In [ ]:
sns.barplot(x = "Destination", y = "Transported", data = train_data)

In [ ]:
train_data["Destination"].value_counts()

In [ ]:
train_data.groupby("Transported")["Destination"].value_counts()

In [ ]:
train_data.groupby("Transported")["Age"].describe()

In [ ]:
train_data[train_data["VIP"] == True]["HomePlanet"].value_counts()

In [ ]:
train_data[train_data["VIP"] == True]["CryoSleep"].value_counts()

In [ ]:
train_data[train_data["VIP"] == True]["Destination"].value_counts()

In [ ]:
train_data[train_data["CryoSleep"] == True]["Age"].describe()

In [ ]:
train_data[train_data["Transported"] == 1]["Age"].describe()

In [ ]:
train_data[train_data["Transported"] == 0]["Age"].describe()

In [ ]:
train_data.groupby("HomePlanet")["CryoSleep"].value_counts()

In [ ]:
train_data.groupby("Destination")["CryoSleep"].value_counts()

<div style="border-radius:10px; border:#808080 solid; padding: 15px; background-color: ##F0E68C ; font-size:100%; text-align:left">

<h3 align="left"><font color=blue>Observe</font></h3>

* **199 VIP passengers 76 of them arrived. 4198 of 8291 non-VIP passengers arrived. According to the ratio, VIP is not very effective in the dependent variable**
* **4602 HomePlanet Earth passengers, 1951 of them arrived.2131 HomePlanet Europa passengers, 1404 of them arrived.1759 HomePlanet Mars passengers, 920 of them arrived. HP becoming Europa effective in the dependent variable**
* **3037 sleeping passengers 2483 of them arrived. 5439 not sleeping passengers 1789 of them arrived. Sleeping very effective in the dependent variable**
* **5915 Destination Trappist passengers, 2787 of them arrived.1800 Destination 55 Cancri e passengers, 1098 of them arrived.796 Destination PSO passengers, 401 of them arrived.**
* **Age feature not effective in the dependent variable**
* **VIP passengers not have Earth HomePlanet**
* **199 VIP passangers 2 of them sleeping**

<a id = "4"></a>
# Missing Values

In [ ]:
total_data = pd.concat([train_data, test_data], ignore_index=True)

In [ ]:
total_data

In [ ]:
mss.matrix(total_data)

In [ ]:
mss.heatmap(total_data)

In [ ]:
total_data.isnull().sum()

<a id = "5"></a>
## Name and Cabin

In [ ]:
total_data[
    (total_data["Cabin"].isnull() == True) &
    (total_data["Name"].isnull() == True)]

In [ ]:
total_data.head(10)

In [ ]:
total_data["Cabin"].isnull().sum()

<div style="border-radius:10px; border:#808080 solid; padding: 15px; background-color: ##F0E68C ; font-size:100%; text-align:left">

<h3 align="left"><font color=blue>Observe</font></h3>

* **Surname same passengers have same cabins. According to this property missing values can fill each other**

In [ ]:
name_list = list(total_data["Name"].values)

In [ ]:
name_list

In [ ]:
surname_list = list()
for i in name_list:
    if type(i) != float:
        surname_list.append(i.split()[1])
    else:
        surname_list.append(np.nan)

In [ ]:
surname_list

In [ ]:
total_data.drop(["Name"],axis = 1, inplace = True)

In [ ]:
total_data["Surname"] = surname_list

In [ ]:
total_data

In [ ]:
total_data[total_data["Cabin"].isnull() == True]

In [ ]:
cabin_index = total_data[total_data["Cabin"].isnull() == True]["Surname"].index

In [ ]:
sur_list = list(total_data[total_data["Cabin"].isnull() == True]["Surname"].values)

In [ ]:
sur_list

In [ ]:
cabin_dict = dict()
for i in sur_list:
    if type(i) != float:
        
        list_ = list(total_data[total_data["Surname"] == i]["Cabin"].values)
        a = max(set(list_), key = list_.count)
        cabin_dict[i] = a

In [ ]:
cabin_dict

In [ ]:
for i in cabin_index:
    sur = total_data.iloc[i]["Surname"]
    try:
        total_data["Cabin"][i] = cabin_dict[sur]
    except:
        pass

In [ ]:
#check
total_data["Cabin"].isnull().sum()

In [ ]:
total_data[total_data["Surname"].isnull() == True]

In [ ]:
surname_index = total_data[total_data["Surname"].isnull() == True]["Cabin"].index

In [ ]:
cabin_list = list(total_data[total_data["Surname"].isnull() == True]["Cabin"].values)

In [ ]:
sur_dict = dict()
for i in cabin_list:
    if type(i) != float:        
        list_ = list(total_data[total_data["Cabin"] == i]["Surname"].values)
        a = max(set(list_), key = list_.count)
        sur_dict[i] = a

In [ ]:
sur_dict

In [ ]:
for i in surname_index:
    cab = total_data.iloc[i]["Cabin"]
    try:
        total_data["Surname"][i] = sur_dict[cab]
    except:
        pass

In [ ]:
total_data["Surname"].isnull().sum()

In [ ]:

cabin_index = total_data[total_data["Cabin"].isnull() == True]["Surname"].index
sur_list = list(total_data[total_data["Cabin"].isnull() == True]["Surname"].values)
cabin_dict = dict()
for i in sur_list:
    if type(i) != float:
        
        list_ = list(total_data[total_data["Surname"] == i]["Cabin"].values)
        a = max(set(list_), key = list_.count)
        cabin_dict[i] = a
for i in cabin_index:
    sur = total_data.iloc[i]["Surname"]
    try:
        total_data["Cabin"][i] = cabin_dict[sur]
    except:
        pass

In [ ]:
total_data["Cabin"].isnull().sum()

In [ ]:
surname_index = total_data[total_data["Surname"].isnull() == True]["Cabin"].index
cabin_list = list(total_data[total_data["Surname"].isnull() == True]["Cabin"].values)
sur_dict = dict()
for i in cabin_list:
    if type(i) != float:        
        list_ = list(total_data[total_data["Cabin"] == i]["Surname"].values)
        a = max(set(list_), key = list_.count)
        sur_dict[i] = a
for i in surname_index:
    cab = total_data.iloc[i]["Cabin"]
    try:
        total_data["Surname"][i] = sur_dict[cab]
    except:
        pass

In [ ]:
total_data["Cabin"].isnull().sum()

In [ ]:
total_data["Surname"].isnull().sum()

In [ ]:
C1 = list()
C2 = list()
for i in total_data["Cabin"].values:
    if (type(i)!= float):
        i = i.split("/")
        C1.append(i[0])
        C2.append(i[2])
    else:
        C1.append(np.nan)
        C2.append(np.nan)

In [ ]:
total_data["C1"] = C1
total_data["C2"] = C2
total_data.drop(["Cabin"], axis = 1, inplace = True)

In [ ]:
total_data

<a id = "6"></a>
## PassengerId

In [ ]:
id_ = total_data["PassengerId"].values

In [ ]:
id_list = list()
for i in id_:
    i = i.split("_")
    id_list.append(i[1])

In [ ]:
id_list

In [ ]:
total_data.drop(["PassengerId"], axis = 1, inplace=True)

In [ ]:
total_data["Id"] = id_list

In [ ]:
total_data

<a id = "7"></a>
## Luxury

In [ ]:
# Sleeping or under 13 age passengers can't spend money. There fore fill 0

In [ ]:
food_index = total_data[(total_data["FoodCourt"].isnull() == True) & ((
total_data["CryoSleep"] == True ) | total_data["Age"]<13)].index
room_index = total_data[(total_data["RoomService"].isnull() == True) & ((
total_data["CryoSleep"] == True ) | total_data["Age"]<13)].index
shop_index = total_data[(total_data["ShoppingMall"].isnull() == True) & ((
total_data["CryoSleep"] == True ) | total_data["Age"]<13)].index
spa_index = total_data[(total_data["Spa"].isnull() == True) & ((
total_data["CryoSleep"] == True ) | total_data["Age"]<13)].index
vr_index = total_data[(total_data["VRDeck"].isnull() == True) & ((
total_data["CryoSleep"] == True ) | total_data["Age"]<13)].index

In [ ]:
total_data["FoodCourt"][food_index] = 0
total_data["RoomService"][room_index] = 0
total_data["Spa"][spa_index] = 0
total_data["ShoppingMall"][shop_index] = 0
total_data["VRDeck"][vr_index] = 0

In [ ]:
total_data.isnull().sum()

In [ ]:
luxury = (total_data["FoodCourt"] + total_data["RoomService"] +  total_data["ShoppingMall"] + 
 total_data["Spa"] + total_data["VRDeck"]) / 5

In [ ]:
luxury = luxury.values

In [ ]:
luxury

In [ ]:
total_data["Luxury"] = luxury

In [ ]:
total_data.drop([
    "FoodCourt", "ShoppingMall", "Spa", "VRDeck", "RoomService"
], axis = 1, inplace = True)

In [ ]:
total_data

In [ ]:
total_data.isnull().sum()

<a id = "8"></a>
## HomePlanet

<div style="border-radius:10px; border:#808080 solid; padding: 15px; background-color: ##F0E68C ; font-size:100%; text-align:left">

<h3 align="left"><font color=blue>Observe</font></h3>

* **Surname same passengers have same homeplanet. According to this property missing values can fill**

In [ ]:
hp_index = total_data[total_data["HomePlanet"].isnull() == True]["Surname"].index
sur_list = list(total_data[total_data["HomePlanet"].isnull() == True]["Surname"].values)
hp_dict = dict()
for i in sur_list:
    if type(i) != float:
        
        list_ = list(total_data[total_data["Surname"] == i]["HomePlanet"].values)
        a = max(set(list_), key = list_.count)
        hp_dict[i] = a
for i in hp_index:
    sur = total_data.iloc[i]["Surname"]
    try:
        total_data["HomePlanet"][i] = hp_dict[sur]
    except:
        pass

In [ ]:
total_data["HomePlanet"].isnull().sum()

In [ ]:
total_data[total_data["HomePlanet"].isnull() == True]

In [ ]:
train_data[
    (train_data["Destination"] == "55 Cancri e") & (train_data["VIP"] == False)
    &(train_data["CryoSleep"] == True) &
    ((train_data["Age"]>=13) | (train_data["Age"]<=54))
    
]["HomePlanet"].value_counts()

In [ ]:
train_data[
    (train_data["Destination"] == "PSO J318.5-22") & (train_data["VIP"] == False)
    &(train_data["CryoSleep"] == True) &
    ((train_data["Age"]>=13) | (train_data["Age"]<=54))
    
]["HomePlanet"].value_counts()

In [ ]:
train_data[
    (train_data["Destination"] == "TRAPPIST-1e") & (train_data["VIP"] == False)
    &(train_data["CryoSleep"] == False) &
    ((train_data["Age"]>=13) | (train_data["Age"]<=54))
    
]["HomePlanet"].value_counts()

In [ ]:
total_data[(total_data["HomePlanet"].isnull() == True) &
           (total_data["Destination"] == "55 Cancri e")]

In [ ]:
total_data["HomePlanet"][[234,807,10238]] = "Europa"

In [ ]:
hom_index = total_data[total_data["HomePlanet"].isnull() == True].index

In [ ]:
total_data["HomePlanet"][hom_index] = "Earth"

In [ ]:
total_data["HomePlanet"].isnull().sum()

<a id = "9"></a>
## Destination

In [ ]:
dest_index = total_data[total_data["Destination"].isnull() == True]["Surname"].index
dest_list = list(total_data[total_data["Destination"].isnull() == True]["Surname"].values)
dest_dict = dict()
for i in dest_list:
    if type(i) != float:
        
        list_ = list(total_data[total_data["Surname"] == i]["Destination"].values)
        a = max(set(list_), key = list_.count)
        dest_dict[i] = a
for i in dest_index:
    sur = total_data.iloc[i]["Surname"]
    try:
        total_data["Destination"][i] = dest_dict[sur]
    except:
        pass

In [ ]:
total_data[total_data["Destination"].isnull() == True]

In [ ]:
train_data[
    (train_data["HomePlanet"] == "Mars") & (train_data["VIP"] == False)
    &(train_data["CryoSleep"] == False) & 
    ((train_data["Age"]>=20) | (train_data["Age"]<=50))
    
]["Destination"].value_counts()

In [ ]:
total_data["Destination"].fillna(value ="TRAPPIST-1e", inplace=True)

In [ ]:
total_data["Destination"].isnull().sum()

In [ ]:
total_data.isnull().sum()

<a id = "10"></a>
## VIP

In [ ]:
vıp_index = total_data[total_data["VIP"].isnull() == True]["Surname"].index
vıp_list = list(total_data[total_data["VIP"].isnull() == True]["Surname"].values)
vıp_dict = dict()
for i in vıp_list:
    if type(i) != float:
        
        list_ = list(total_data[total_data["Surname"] == i]["VIP"].values)
        a = max(set(list_), key = list_.count)
        vıp_dict[i] = a
for i in vıp_index:
    sur = total_data.iloc[i]["Surname"]
    try:
        total_data["VIP"][i] = vıp_dict[sur]
    except:
        pass

In [ ]:
total_data[total_data["VIP"].isnull() == True]

In [ ]:
train_data[train_data["VIP"] == True]["HomePlanet"].value_counts()

In [ ]:
total_data["VIP"][[12345,4824]] = False

In [ ]:
total_data[(total_data["HomePlanet"] == "Earth")]["VIP"].value_counts()

In [ ]:
train_data[
    (train_data["HomePlanet"] == "Europa") & (train_data["Destination"] =="55 Cancri e" )&
    (train_data["CryoSleep"] == True) & 
    ((train_data["Age"]>=19) | (train_data["Age"]<=63) & (train_data["Transported"]==1))
    
]["VIP"].value_counts()

In [ ]:
total_data["VIP"].fillna(value = False, inplace = True)

In [ ]:
total_data[total_data["VIP"].isnull() == True]

In [ ]:
total_data.isnull().sum()

<a id = "11"></a>
## CRYOSLEEP

In [ ]:
slp_index = total_data[total_data["CryoSleep"].isnull() == True]["Surname"].index
slp_list = list(total_data[total_data["CryoSleep"].isnull() == True]["Surname"].values)
slp_dict = dict()
for i in slp_list:
    if type(i) != float:
        
        list_ = list(total_data[total_data["Surname"] == i]["CryoSleep"].values)
        a = max(set(list_), key = list_.count)
        slp_dict[i] = a
for i in slp_index:
    sur = total_data.iloc[i]["Surname"]
    try:
        total_data["CryoSleep"][i] = slp_dict[sur]
    except:
        pass

In [ ]:
total_data[total_data["CryoSleep"].isnull() == True]

In [ ]:
train_data[
    (train_data["HomePlanet"] == "Earth") & (train_data["Destination"] =="PSO J318.5-22" )&
    (train_data["VIP"] == False) & 
    ((train_data["Age"]<=51) )
    
]["CryoSleep"].value_counts()

In [ ]:
total_data["CryoSleep"][[5531,9837,12230, 5702]] = True

In [ ]:
total_data["CryoSleep"].fillna(value = False, inplace = True)

In [ ]:

total_data["CryoSleep"].isnull().sum()

In [ ]:
total_data.isnull().sum()

<a id = "12"></a>
## AGE

In [ ]:
age_index = total_data[(total_data["Age"].isnull()==True) & (total_data["Luxury"] == 0) & 
          (total_data["CryoSleep"] == False)].index

In [ ]:
train_data[
    (train_data["VRDeck"] == 0) & (train_data["ShoppingMall"] == 0)
    & (train_data["RoomService"] == 0) & (train_data["Spa"] == 0) &
    (train_data["FoodCourt"] == 0) & (train_data["CryoSleep"] == False)
]["Age"].median()

In [ ]:
total_data["Age"][age_index] = 7

In [ ]:
train_data["Age"].median()

In [ ]:
total_data["Age"].fillna(27, inplace=True)

In [ ]:
total_data["Age"].isnull().sum()

In [ ]:
total_data.isnull().sum()

<a id = "13"></a>
## C1-C2

In [ ]:
cab = train_data["Cabin"].values
cab = list(cab)
cab1 = list()
cab2 = list()
for i in cab:
    if (type(i) != float):
        i = i.split("/")
        cab1.append(i[0])
        cab2.append(i[2])
    else:
        cab1.append(np.nan)
        cab2.append(np.nan)

In [ ]:
train_data["C1"] = cab1
train_data["C2"] = cab2

In [ ]:
sns.barplot(x = "C1", y= "Transported", data = train_data)

In [ ]:
train_data.groupby("Transported")["C1"].value_counts()

In [ ]:
train_data["C1"].value_counts()

In [ ]:
sns.barplot(x = "C2", y= "Transported", data = train_data)

In [ ]:
train_data.groupby("Transported")["C2"].value_counts()

In [ ]:
train_data["C2"].value_counts()

In [ ]:
train_surname = list()
for i in train_data["Name"].values:
    if(type(i)!=float):
        i = i.split(" ")
        train_surname.append(i[1])
    else:
        train_surname.append(np.nan)
    

In [ ]:
train_data["Surname"] = train_surname

In [ ]:
C1_index = total_data[total_data["C1"].isnull() == True]["Surname"].index
C1_list = list(total_data[total_data["C1"].isnull() == True]["Surname"].values)
C1_dict = dict()
for i in C1_list:
    if type(i) != float:
        
        list_ = list(total_data[total_data["Surname"] == i]["C1"].values)
        a = max(set(list_), key = list_.count)
        C1_dict[i] = a
for i in C1_index:
    sur = total_data.iloc[i]["Surname"]
    try:
        total_data["C1"][i] = C1_dict[sur]
    except:
        pass

In [ ]:
total_data["C1"].isnull().sum()

In [ ]:
C2_index = total_data[total_data["C2"].isnull() == True]["Surname"].index
C2_list = list(total_data[total_data["C2"].isnull() == True]["Surname"].values)
C2_dict = dict()
for i in C2_list:
    if type(i) != float:
        
        list_ = list(total_data[total_data["Surname"] == i]["C2"].values)
        a = max(set(list_), key = list_.count)
        C2_dict[i] = a
for i in C2_index:
    sur = total_data.iloc[i]["Surname"]
    try:
        total_data["C2"][i] = C2_dict[sur]
    except:
        pass

In [ ]:
total_data["C2"].isnull().sum()

In [ ]:
total_data["C1"].value_counts()

In [ ]:
total_data["C2"].value_counts()

In [ ]:
total_data[total_data["C1"].isnull() == True]

In [ ]:
total_data["C1"][7244] = "C"
total_data["C1"][7330] = "F"
total_data["C1"][11940] = "F"
total_data["C1"][10717] = "D"

In [ ]:
total_data[total_data["C1"].isnull() == True]


In [ ]:
train_data[(train_data["C2"] == "S") & (train_data["VIP"] == False) &
          (train_data["Destination"] =="TRAPPIST-1e")  & (train_data["CryoSleep"] == False)
          & (train_data["HomePlanet"] == "Europa")
           & (train_data["Age"] == 29) & (train_data["Transported"] == 1)]["C1"].value_counts()

In [ ]:
train_data[(train_data["VIP"] == False) &
          (train_data["Destination"] =="TRAPPIST-1e")  & (train_data["CryoSleep"] == False)
          & (train_data["HomePlanet"] == "Mars")
           & (train_data["Transported"] == 0)]["C1"].value_counts()

In [ ]:

total_data["C1"][[3701,1980]] = "C"

In [ ]:
total_data["C1"].fillna("F", inplace = True)

In [ ]:
total_data["C1"].isnull().sum()

In [ ]:
total_data["C1"].value_counts()

In [ ]:
total_data[total_data["C2"].isnull() == True]

In [ ]:
train_data[(train_data["HomePlanet"] == "Mars") & (train_data["CryoSleep"] == True) &
          (train_data["VIP"] == False) & (train_data["Destination"] == "TRAPPIST-1e")]["C2"].value_counts()

In [ ]:
total_data["C2"][[12941, 12942]] = "S"

In [ ]:
total_data["C2"].fillna("P", inplace = True)

In [ ]:
total_data["C2"].value_counts()

In [ ]:
total_data["C2"].isnull().sum()

<a id = "14"></a>
## Surname Drop

In [ ]:
total_data.drop(["Surname"], axis = 1, inplace = True)

In [ ]:
total_data.drop(["Id"], axis = 1, inplace = True)

In [ ]:
total_data.isnull().sum()

<a id = "15"></a>
## Reducing Feature and Converting

In [ ]:
total_data

In [ ]:
list_ = ["HomePlanet", "CryoSleep", "Destination", "VIP", "C1", "C2"]
for i in list_:
    lbe = LabelEncoder()
    total_data[i] = lbe.fit_transform(total_data[i])


In [ ]:
total_data

<a id = "16"></a>
## Outlier Samples

In [ ]:
q1_age = total_data["Age"].quantile(0.25)

In [ ]:
q1_age

In [ ]:
q3_age = total_data["Age"].quantile(0.75)

In [ ]:
q3_age

In [ ]:
IQR = q3_age-q1_age

In [ ]:
IQR

In [ ]:
down_threshold = q1_age-1.5*IQR
up_threshold = q3_age + 1.5*IQR

In [ ]:
down_threshold

In [ ]:
up_threshold

In [ ]:
age_index = total_data[total_data["Age"]>up_threshold].index

In [ ]:
total_data["Age"][age_index] = up_threshold

In [ ]:
total_data["Age"].max()

In [ ]:
wr_index = total_data[(total_data["CryoSleep"] == 1) & (total_data["Luxury"]!=0)].index

In [ ]:
total_data["CryoSleep"][wr_index] = 0

In [ ]:
q1_lux = total_data[total_data["Luxury"]!=0]["Luxury"].quantile(0.25)

In [ ]:
q1_lux

In [ ]:
q3_lux = total_data[total_data["Luxury"]!=0]["Luxury"].quantile(0.75)

In [ ]:
q3_lux

In [ ]:
IQR2 = q3_lux-q1_lux

In [ ]:
IQR2

In [ ]:
down_threshold2 = q1_lux-1.5*IQR2
up_threshold2 = q3_lux + 1.5*IQR2

In [ ]:
down_threshold2

In [ ]:
up_threshold2

In [ ]:
lux_index = total_data[total_data["Luxury"]>up_threshold2].index

In [ ]:
total_data["Luxury"][lux_index] = up_threshold2

In [ ]:
total_data[total_data["Luxury"]>up_threshold2]

<a id = "17"></a>
## Normalize

In [ ]:
total_data = (total_data-np.min(total_data)) / (np.max(total_data)-(np.min(total_data)))

In [ ]:
total_data

<a id = "18"></a>
## Train-Test Split for Return to original Dataset

In [ ]:
train_data

In [ ]:
train_data = total_data.iloc[:8693]

In [ ]:
train_data

In [ ]:
test_data = total_data.iloc[8693:]

In [ ]:
test_data.index = range(0,4277)

In [ ]:
test_data

<a id = "19"></a>
# MODEL

<a id = "20"></a>
## Dependent and Undependent Variable

In [ ]:
X = train_data.drop(["Transported"], axis = 1)

In [ ]:
X.shape

In [ ]:
y = train_data["Transported"]

In [ ]:
y.shape

<a id = "21"></a>
## Train-Test Split For Model

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X,y, test_size=0.25,random_state=42)

In [ ]:
X_test.shape

In [ ]:
y_train.shape

In [ ]:
y_test.shape

In [ ]:
y_train.value_counts()

In [ ]:
y_test.value_counts()

<a id = "22"></a>
## LGB

In [ ]:
lgb_model = LGBMClassifier(random_state=42,
                           min_split_gain=0.007,                       
                           num_leaves=15,
                           learning_rate=0.05,                           
                           n_estimators=100,                         
                           max_depth=7,
                           subsample=0.8,                          
                       
                          force_row_wise=True)

In [ ]:
lgb_model.fit(X_train, y_train)

In [ ]:
y_pred = lgb_model.predict(X_test)

In [ ]:
accuracy_score(y_test, y_pred)